In [ ]:
channels = ['ndtv', 'indiatoday', 'wion']
data = {}

for c in (channels):
  with open("/content/" + c + ".txt", "rb") as file:
    data[c] = [file.read().decode("utf-8") ]
    

In [ ]:
import pandas as pd
pd.set_option('max_colwidth',150)
data_df = pd.DataFrame.from_dict(data).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

In [ ]:
data_df.transcript.loc['ndtv']

In [ ]:
import re
import string
def clean_text_round1(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '',text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [ ]:
round1 = lambda x: clean_text_round1(x)
data_clean = pd.DataFrame(data_df.transcript.apply(round1))

In [ ]:
def clean_text_round2(text):
  text = re.sub('[‘’“”...]', '', text)
  text = re.sub('\n', '', text)
  return text

In [ ]:
round2 = lambda x: clean_text_round2(x)
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_df.to_pickle("corpus.pkl")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(),
columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [ ]:
import pickle
data_dtm.to_pickle("dtm.pkl")
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

In [ ]:
import pandas as pd
data = pd.read_pickle('dtm.pkl')
data = data.transpose()
data.head()

In [ ]:
top_dict = {}
for c in data.columns:
  top = data[c].sort_values(ascending=False).head(30)
  top_dict[c]= list(zip(top.index, top.values))
top_dict

In [ ]:
for channel, top_words in top_dict.items():
  print(channel)
  print(', '.join([word for word, count in top_words[0:20]]))
  print('---')

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
add_stop_words=stop_words
add_stop_words

In [ ]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
ata_clean = pd.read_pickle('data_clean.pkl')
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.transcript)
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = data_clean.index
import pickle
pickle.dump(cv, open("cv_stop.pkl", "wb"))
data_stop.to_pickle("dtm_stop.pkl")

EDA


In [ ]:
from wordcloud import WordCloud
wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
max_font_size=150, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 6]
full_names = ['ndtv', 'indiatoday', 'wion']
for index, comedian in enumerate(data.columns):
  wc.generate(data_clean.transcript[comedian])
  plt.subplot(3, 4, index+1)
  plt.imshow(wc, interpolation="bilinear")
  plt.axis("off")
  plt.title(full_names[index])
plt.show()

In [ ]:
import pandas as pd
data = pd.read_pickle('corpus.pkl')
data

In [ ]:
from textblob import TextBlob
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
data['polarity'] = data['transcript'].apply(pol)
data['subjectivity'] = data['transcript'].apply(sub)
data

Sentiment analysis

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
for index, channel in enumerate(data.index):
  x = data.polarity.loc[channel]
  y = data.subjectivity.loc[channel]
  plt.scatter(x, y, color='blue')
  plt.text(x+.001, y+.001, channel, fontsize=10)
  plt.xlim(-.01, .30)
plt.title('Sentiment Analysis', fontsize=20)
plt.xlabel('<-- Negative -------- Positive -->', fontsize=15)
plt.ylabel('<-- Facts -------- Opinions -->', fontsize=15)
plt.show()

In [ ]:
import numpy as np
import math
def split_text(text, n=10):

    length = len(text)
    size = math.floor(length / n)
    start = np.arange(0, length, size)
    split_list = []
    for piece in range(n):
      split_list.append(text[start[piece]:start[piece]+size])
      return split_list

In [ ]:
data

In [1]:
list_pieces = []
for t in data.transcript:
  split = split_text(t)
  list_pieces.append(split)
list_pieces

NameError: ignored

Topic modelling

In [ ]:
import pandas as pd
import pickle
data = pd.read_pickle('dtm_stop.pkl')
data

In [ ]:
from gensim import matutils, models
import scipy.sparse

In [ ]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=10)
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()


In [ ]:
from nltk import word_tokenize, pos_tag

In [ ]:
def nouns_adj(text):
  '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
  is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
  tokenized = word_tokenize(text)
  nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
  return ' '.join(nouns_adj)


In [ ]:
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


In [ ]:
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

In [ ]:
from nltk.corpus import stopwords
add_stop_words = set(stopwords.words('english'))

In [ ]:
# Create a new document-term matrix using only nouns and adjectives
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
# Re-add the additional stop words since we are recreating the document-term matrix
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
# Recreate a document-term matrix with only nouns and adjectives
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))


In [ ]:
# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna,
passes=10)
ldana.print_topics()

In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna,
passes=10)
ldana.print_topics()


In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna,
passes=10)
ldana.print_topics()


In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna,
passes=80)
ldana.print_topics()

In [ ]:
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))